# CoT-Structed_Output-Checklist-RAG

In [1]:
%pip install -q chromadb openai pydantic python-dotenv sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:

import logging
import os
from typing import Any, Dict, List, Literal, Optional, Union

import chromadb
import orjson
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from openai import AsyncOpenAI
from pydantic import BaseModel, Field, ValidationError

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

load_dotenv()

VLLM_MODEL_NAME = "Qwen/Qwen2.5-14B-Instruct"
VLLM_BASE_URL = os.getenv("VLLM_BASE_URL")
VLLM_API_KEY = os.getenv("VLLM_API_KEY")
CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = os.getenv("CHROMA_PORT")
CHROMA_COLLECTION_NAME = os.getenv("CHROMA_COLLECTION_NAME")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")

chroma_client = await chromadb.AsyncHttpClient(host=CHROMA_HOST, port=CHROMA_PORT)
vllm_client = AsyncOpenAI(base_url=VLLM_BASE_URL, api_key=VLLM_API_KEY)
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBEDDING_MODEL_NAME)

display(VLLM_BASE_URL)

2024-10-10 21:07:35,877 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-10-10 21:07:35,901 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-10-10 21:07:35,920 - INFO - HTTP Request: GET http://localhost:8000/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
2024-10-10 21:07:35,925 - INFO - HTTP Request: GET http://localhost:8000/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"
/home/michael/Github/nfd-rag-2024/.micromamba/envs/default/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-10 21:07:38,545 - INFO - Load pretrained SentenceTransformer: cointegrated/LaBSE-en-ru


'http://154.20.254.95:50856/v1'

In [3]:
class DocumentAnalysis(BaseModel):
    is_relevant: bool = Field(..., description="Релевантен ли документ запросу?")
    key_information: str = Field(..., description="Ключевая информация из документа")

class QueryAnalysis(BaseModel):
    query_type: Literal["factual", "procedural", "conceptual", "other"] = Field(..., description="Тип запроса")
    main_topic: str = Field(..., description="Основная тема запроса")
    required_info: List[str] = Field(..., description="Список необходимой информации для ответа")

class RetrievalStrategy(BaseModel):
    query_type: Literal["semantic", "keyword", "hybrid"] = Field(..., description="Тип поиска для извлечения документов")
    top_k: int = Field(..., description="Количество документов для извлечения")
    filter_criteria: Optional[Dict[str, Any]] = Field(None, description="Критерии фильтрации документов")

class AnswerFormulation(BaseModel):
    main_points: List[str] = Field(..., description="Основные пункты ответа")
    additional_context: Optional[str] = Field(None, description="Дополнительный контекст")
    confidence_level: Literal["high", "medium", "low"] = Field(..., description="Уровень уверенности в ответе")

class ResponseModel(BaseModel):
    query_analysis: QueryAnalysis
    document_analysis: List[DocumentAnalysis]
    retrieval_strategy: RetrievalStrategy
    answer_formulation: AnswerFormulation
    final_answer: str
    sources: List[str]

async def retrieve_documents(query: str, strategy: RetrievalStrategy) -> List[dict]:
    collection = await chroma_client.get_or_create_collection(
        name=CHROMA_COLLECTION_NAME,
        embedding_function=embedding_function,
    )
    results = await collection.query(
        query_texts=[query],
        n_results=strategy.top_k,
        where=strategy.filter_criteria,
    )
    return [
        {"id": id, "content": doc, "metadata": meta}
        for id, doc, meta in zip(
            results["ids"][0], results["documents"][0], results["metadatas"][0]
        )
    ]


RESPONSE_GENERATION_PROMPT = '''Вопрос пользователя: {query}

Контекст из документов:
{context}

Проанализируйте вопрос и контекст, затем сформулируйте ответ, следуя структуре:

1. Анализ запроса
2. Анализ документов
3. Стратегия поиска
4. Формулировка ответа
5. Итоговый ответ
6. Источники

Ваш ответ должен быть в следующем формате JSON:

{{
    "query_analysis": {{
        "query_type": "factual" | "procedural" | "conceptual" | "other",
        "main_topic": "<основная тема запроса>",
        "required_info": ["<необходимая информация 1>", "<необходимая информация 2>", ...]
    }},
    "document_analysis": [
        {{
            "is_relevant": true | false,
            "key_information": "<ключевая информация из документа>"
        }},
        ...
    ],
    "retrieval_strategy": {{
        "query_type": "semantic" | "keyword" | "hybrid",
        "top_k": <число>,
        "filter_criteria": null
    }},
    "answer_formulation": {{
        "main_points": ["<основной пункт 1>", "<основной пункт 2>", ...],
        "additional_context": "<дополнительный контекст или null>",
        "confidence_level": "high" | "medium" | "low"
    }},
    "final_answer": "<краткий и точный ответ на вопрос>",
    "sources": ["<номер документа>", ...]
}}

Пожалуйста, убедитесь, что ваш ответ строго соответствует этой структуре JSON.'''

async def generate_response(query: str, documents: List[dict]) -> Union[ResponseModel, str]:
    context = "\n".join([f"Document {i+1}: {doc['content']}" for i, doc in enumerate(documents)])
    prompt = RESPONSE_GENERATION_PROMPT.format(query=query, context=context)

    try:
        response = await vllm_client.chat.completions.create(
            model=VLLM_MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
            max_tokens=2048,
        )

        response_content = response.choices[0].message.content
        response_data = orjson.loads(response_content)

        # Проверяем наличие всех необходимых ключей
        required_keys = ["query_analysis", "document_analysis", "retrieval_strategy", "answer_formulation", "final_answer", "sources"]
        if not all(key in response_data for key in required_keys):
            missing_keys = [key for key in required_keys if key not in response_data]
            raise ValueError(f"В ответе LLM отсутствуют следующие ключи: {', '.join(missing_keys)}")

        return ResponseModel(**response_data)
    except orjson.JSONDecodeError as e:
        logger.error(f"Ошибка при разборе JSON: {e}")
        logger.error(f"Содержимое ответа: {response_content}")
        return "Ошибка при разборе ответа: неверный формат JSON"
    except ValidationError as e:
        logger.error(f"Ошибка валидации: {e}")
        return f"Ошибка при валидации ответа: {str(e)}"
    except Exception as e:
        logger.error(f"Неожиданная ошибка: {e}")
        return f"Произошла неожиданная ошибка: {str(e)}"

async def rag_assistant(query: str) -> str:
    initial_strategy = RetrievalStrategy(query_type="semantic", top_k=5)
    documents = await retrieve_documents(query, initial_strategy)

    response = await generate_response(query, documents)

    if isinstance(response, str):
        return response  # Возвращаем сообщение об ошибке

    final_response = f"""
    Анализ запроса:
    - Тип запроса: {response.query_analysis.query_type}
    - Основная тема: {response.query_analysis.main_topic}
    - Требуемая информация: {', '.join(response.query_analysis.required_info)}

    Анализ документов:
    {' '.join([f"Документ {i+1}: {'Релевантный' if doc.is_relevant else 'Нерелевантный'}" for i, doc in enumerate(response.document_analysis)])}

    Стратегия поиска:
    - Тип поиска: {response.retrieval_strategy.query_type}
    - Количество документов: {response.retrieval_strategy.top_k}
    - Фильтры: {response.retrieval_strategy.filter_criteria}

    Формулировка ответа:
    - Основные пункты: {', '.join(response.answer_formulation.main_points)}
    - Дополнительный контекст: {response.answer_formulation.additional_context}
    - Уровень уверенности: {response.answer_formulation.confidence_level}

    Итоговый ответ: {response.final_answer}

    Источники информации: {', '.join(response.sources)}
    """

    return final_response

In [4]:
# Пример использования
query = "Какая тема занятия номер 2?"
answer = await rag_assistant(query)
print(answer)

2024-10-10 21:07:40,812 - INFO - HTTP Request: POST http://localhost:8000/api/v1/collections?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-10 21:07:40,934 - INFO - HTTP Request: POST http://localhost:8000/api/v1/collections/c6db97b8-8ea7-466b-81a1-eea76c40a6d6/query "HTTP/1.1 200 OK"
2024-10-10 21:08:52,774 - INFO - HTTP Request: POST http://154.20.254.95:50856/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-10 21:08:52,778 - ERROR - Неожиданная ошибка: В ответе LLM отсутствуют следующие ключи: query_analysis, document_analysis, retrieval_strategy, answer_formulation, final_answer, sources


Произошла неожиданная ошибка: В ответе LLM отсутствуют следующие ключи: query_analysis, document_analysis, retrieval_strategy, answer_formulation, final_answer, sources
